In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# =========================
# Paths
# =========================
model_path = "/content/drive/MyDrive/transfer_learning_model_Xception/best_model.keras"  # your best model
img_path   = "/content/6.png"  # replace with your image

# =========================
# Load Model
# =========================
model = keras.models.load_model(model_path)

# =========================
# Preprocess Image
# =========================
IMG_SIZE = (256, 256)

img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # make batch of 1
img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)

# =========================
# Predict
# =========================
pred = model.predict(img_array)
class_index = np.argmax(pred, axis=1)[0]
confidence = pred[0][class_index] * 100

# =========================
# Map class index to label
# =========================
class_labels = {0: "fake", 1: "real"}  # make sure this matches your dataset
predicted_label = class_labels[class_index]

print(f"Prediction: {predicted_label} ({confidence:.2f}%)")


In [ ]:
# All-in-one: extract @2 FPS, preprocess correctly (256x256), predict, diagnostics
import os, cv2, numpy as np, tensorflow as tf
from tensorflow.keras.preprocessing import image
from glob import glob
import shutil

# ---------- CONFIG ----------
MODEL_PATH = "/content/drive/MyDrive/transfer_learning_model_Xception/best_model.keras"
VIDEO_PATH = "/content/4.mp4"   # your uploaded video
FRAMES_DIR = "/content/frames_extracted"
FPS_TO_EXTRACT = 2                 # 2 frames per second
IMG_SIZE = (256, 256)               # <-- MUST match training
THRESHOLD = 0.5                     # threshold used for binary decision (not final arbiter)
TOP_K_TO_SHOW = 6                   # show top K most-fake frames as diagnostics
# -----------------------------

# load model
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded:", MODEL_PATH)

# helper: extract frames at target FPS
def extract_frames(video_path, out_dir, fps_target=2):
    os.makedirs(out_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError("Cannot open video: " + video_path)
    src_fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    frame_interval = max(1, int(round(src_fps / fps_target)))
    frame_idx = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if (frame_idx % frame_interval) == 0:
            path = os.path.join(out_dir, f"frame_{saved:05d}.jpg")
            cv2.imwrite(path, frame)            # original BGR saved
            saved += 1
        frame_idx += 1
    cap.release()
    print(f"📸 Extracted {saved} frames at ~{fps_target} FPS (src_fps={src_fps:.2f})")
    return saved

# preprocess + predict one frame -> returns (prob_fake, prob_real)
def predict_frame_probs(img_path):
    # load -> resize -> convert to array
    img = image.load_img(img_path, target_size=IMG_SIZE)
    arr = image.img_to_array(img)             # float32
    # use EfficientNet preprocessing (matches training)
    arr = tf.keras.applications.efficientnet.preprocess_input(arr)
    arr = np.expand_dims(arr, axis=0)
    preds = model.predict(arr, verbose=0)[0]  # shape (2,) for softmax [p_fake, p_real]
    # defensively handle shapes
    if len(preds) == 1:
        # binary single-output sigmoid case (unlikely here)
        p_fake = preds[0]
        p_real = 1.0 - p_fake
    else:
        p_fake, p_real = float(preds[0]), float(preds[1])
    return p_fake, p_real

# run extraction
if os.path.exists(FRAMES_DIR):
    # optional: clear old frames to avoid confusion
    shutil.rmtree(FRAMES_DIR)
num = extract_frames(VIDEO_PATH, FRAMES_DIR, FPS_TO_EXTRACT)
if num == 0:
    raise RuntimeError("No frames extracted — check VIDEO_PATH or video FPS.")

# predict on all frames (batch in groups for speed)
frame_files = sorted(glob(os.path.join(FRAMES_DIR, "*.jpg")))
probs = []
for p in frame_files:
    p_fake, p_real = predict_frame_probs(p)
    probs.append((p, p_fake, p_real))

# stats & aggregation
p_fakes = np.array([x[1] for x in probs])
p_reals = np.array([x[2] for x in probs])

mean_fake = p_fakes.mean()
median_fake = np.median(p_fakes)
mean_real = p_reals.mean()
median_real = np.median(p_reals)

# majority vote by frame (>0.5 => fake)
votes = (p_fakes > 0.5).sum()
vote_frac_fake = votes / len(p_fakes)

print("\n=== Frame-level summary ===")
print("Frames evaluated:", len(p_fakes))
print(f"Mean P(fake): {mean_fake:.4f}   Mean P(real): {mean_real:.4f}")
print(f"Median P(fake): {median_fake:.4f}   Median P(real): {median_real:.4f}")
print(f"Fraction frames voted FAKE (>0.5): {vote_frac_fake:.3f} ({votes}/{len(p_fakes)})")

# decide final: combine metrics (you can pick strategy)
# Strategy: if mean_fake > 0.5 or vote_frac_fake > 0.5 or median_fake > 0.5 => FAKE, else REAL
is_fake_final = (mean_fake > 0.5) or (vote_frac_fake > 0.5) or (median_fake > 0.5)
final_label = "FAKE (AI-generated)" if is_fake_final else "REAL (Camera)"

print("\n===================================")
print("🎥 FINAL VIDEO CLASSIFICATION:", final_label)
print("===================================\n")

# Diagnostic: show top-K frames by P(fake)
sorted_by_fake = sorted(probs, key=lambda x: x[1], reverse=True)
print(f"Top {TOP_K_TO_SHOW} frames with highest P(fake):")
for i, (path, pf, pr) in enumerate(sorted_by_fake[:TOP_K_TO_SHOW]):
    print(f"{i+1:02d}. {os.path.basename(path)}  P(fake)={pf:.4f}  P(real)={pr:.4f}")

# Save per-frame CSV
import csv
csv_out = os.path.join(FRAMES_DIR, "frame_predictions.csv")
with open(csv_out, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["frame", "p_fake", "p_real"])
    for (path, pf, pr) in probs:
        writer.writerow([os.path.basename(path), f"{pf:.6f}", f"{pr:.6f}"])
print("\nSaved per-frame predictions to:", csv_out)
